In [1]:
%run './基本定义.ipynb'

# 1.开始训练模型 transE  win18rr

In [2]:

## 使用transE模型，wn18rr数据集

## 可修改model参数，训练rotate模型
## 可修改data_path参数，指定训练的数据集

base_args={
    'cuda': True,
    'evaluate_train': False,
    'double_entity_embedding': False, 
    'double_relation_embedding': False, 
    'negative_sample_size': 1024, 
    'hidden_dim': 200, 
    'gamma': 6.0, 
    'negative_adversarial_sampling': True, 
    'adversarial_temperature': 0.5, 
    'regularization': 0.0, 
    'test_batch_size': 8, 
    'uni_weight': False, 
    'learning_rate': 0.0005,
    'cpu_num': 10, 
    'comments': '\n', 
}

args={
    'do_train': True, 
    'do_test': True, 
    'data_path': 'data/wn18rr', 
    'model': 'TransE', 
    'batch_size': 512, 
    'max_steps': 40000, ## 根据经验TransE wn18rr 40000
    'warm_up_steps': 20000, 
    'no_save': False, 
    'save_path': './models/TransE_wn18rr_baseline', 
    'save_checkpoint_steps': 10000, 
    'valid_steps': 10000, 
    'log_steps': 2000, 
    'classify_steps': 5000, 
    'test_log_steps': 1000, 
    'nentity': 40943, 
    'nrelation': 11,
    'fake':None,
    'do_valid':False,
    'init_checkpoint': None,
}
args = {**base_args,**args}
log_filename = "train"
set_logger(args, filename=log_filename)
input_data = get_input_data(args)
trainer = BaseTrainer.get_trainer(input_data, args)
kge_model = trainer.kge_model
logging.info('Start Training...')
logging.info(f"args is {args}")
logging.info('learning_rate = %f' % trainer.lr)
for step in range(args['max_steps']):
    trainer.basicTrainStep(step)
trainer.save_model()

## 验证集
logging.info('Evaluating on Valid Dataset...')
metrics = kge_model.test_step(kge_model, input_data.valid_triples, input_data.all_true_triples, args)
log_metrics('Valid', args['max_steps'], metrics)

## 测试集
logging.info('Evaluating on Test Dataset...')
metrics = kge_model.test_step(kge_model, input_data.test_triples, input_data.all_true_triples, args)
log_metrics('Test', args['max_steps'], metrics)

2023-12-19 20:00:12,186 INFO     

2023-12-19 20:00:12,188 INFO     Model: TransE
2023-12-19 20:00:12,189 INFO     Data Path: data/wn18rr
2023-12-19 20:00:12,189 INFO     #entity: 40943
2023-12-19 20:00:12,190 INFO     #relation: 11
2023-12-19 20:00:12,191 INFO     #train: 86835	#valid: 3034	#test: 3134
2023-12-19 20:00:18,633 INFO     Model Parameter Configuration:
2023-12-19 20:00:18,635 INFO     Parameter gamma: torch.Size([1]), require_grad = False
2023-12-19 20:00:18,636 INFO     Parameter embedding_range: torch.Size([1]), require_grad = False
2023-12-19 20:00:18,637 INFO     Parameter entity_embedding: torch.Size([40943, 200]), require_grad = True
2023-12-19 20:00:18,638 INFO     Parameter relation_embedding: torch.Size([11, 200]), require_grad = True
2023-12-19 20:00:18,639 INFO     Start Training...
2023-12-19 20:00:18,641 INFO     args is {'cuda': True, 'evaluate_train': False, 'double_entity_embedding': False, 'double_relation_embedding': False, 'negative_sample_size': 1024, 

# 2.开始训练模型 transE  fb15k-237

In [3]:

args_transE_fb = {
    'cuda': True, 
    'fake': None, 
    'do_train': True, 
    'do_valid': False, 
    'do_test': True, 
    'evaluate_train': False, 
    'data_path': 'data/FB15k-237', 
    'model': 'TransE', 
    'double_entity_embedding': False, 
    'double_relation_embedding': False, 
    'negative_sample_size': 256, 
    'hidden_dim': 200, 
    'gamma': 9.0, 
    'negative_adversarial_sampling': True, 
    'adversarial_temperature': 1.0, 
    'batch_size': 1024, 
    'regularization': 0.0, 
    'test_batch_size': 16, 
    'uni_weight': False, 
    'learning_rate': 0.0005, 
    'cpu_num': 10, 
    'init_checkpoint': None, 
    'max_steps': 20000, ## 经验数据 TransE FB15k-237 20000
    'warm_up_steps': 20000, 
    'no_save': False, 
    'save_path': 'models/TransE_FB15k-237_baseline', 
    'comments': '\n', 
    'save_checkpoint_steps': 10000, 
    'valid_steps': 10000, 
    'log_steps': 2000, 
    'classify_steps': 5000, 
    'test_log_steps': 1000, 
    'nentity': 14541, 
    'nrelation': 237
}
log_filename = "train"
set_logger(args_transE_fb, filename=log_filename)
input_data_transE_fb = get_input_data(args_transE_fb)
trainer_transE_fb = BaseTrainer.get_trainer(input_data_transE_fb, args_transE_fb)
kge_model_tansE_fb = trainer_transE_fb.kge_model
logging.info('Start Training...')
logging.info(f"args is {args_transE_fb}")
logging.info('learning_rate = %f' % trainer_transE_fb.lr)
for step in range(args_transE_fb['max_steps']):
    trainer_transE_fb.basicTrainStep(step)
trainer_transE_fb.save_model()

## 验证集
logging.info('Evaluating on Valid Dataset...')
metrics = kge_model_tansE_fb.test_step(kge_model_tansE_fb, input_data_transE_fb.valid_triples, input_data_transE_fb.all_true_triples, args_transE_fb)
log_metrics('Valid', args_transE_fb['max_steps'], metrics)

## 测试集
logging.info('Evaluating on Test Dataset...')
metrics_transE_fb = kge_model_tansE_fb.test_step(kge_model_tansE_fb, input_data_transE_fb.test_triples, input_data_transE_fb.all_true_triples, args_transE_fb)
log_metrics('Test', args_transE_fb['max_steps'], metrics_transE_fb)

2023-12-19 20:01:37,367 INFO     

2023-12-19 20:01:37,367 INFO     

2023-12-19 20:01:37,369 INFO     Model: TransE
2023-12-19 20:01:37,369 INFO     Model: TransE
2023-12-19 20:01:37,371 INFO     Data Path: data/FB15k-237
2023-12-19 20:01:37,371 INFO     Data Path: data/FB15k-237
2023-12-19 20:01:37,372 INFO     #entity: 14541
2023-12-19 20:01:37,372 INFO     #entity: 14541
2023-12-19 20:01:37,373 INFO     #relation: 237
2023-12-19 20:01:37,373 INFO     #relation: 237
2023-12-19 20:01:37,375 INFO     #train: 272115	#valid: 17535	#test: 20466
2023-12-19 20:01:37,375 INFO     #train: 272115	#valid: 17535	#test: 20466
2023-12-19 20:01:39,593 INFO     Model Parameter Configuration:
2023-12-19 20:01:39,593 INFO     Model Parameter Configuration:
2023-12-19 20:01:39,595 INFO     Parameter gamma: torch.Size([1]), require_grad = False
2023-12-19 20:01:39,595 INFO     Parameter gamma: torch.Size([1]), require_grad = False
2023-12-19 20:01:39,597 INFO     Parameter embedding_range: torch.Size([

# 3. 开始训练模型 rotate  win18rr

In [4]:
args_rotate_wn={
    'cuda': True, 
    'fake': None, 
    'do_train': True, 
    'do_valid': False, 
    'do_test': True, 
    'evaluate_train': False, 
    'data_path': 'data/wn18rr', 
    'model': 'RotatE', 
    'double_entity_embedding': True, 
    'double_relation_embedding': False, 
    'negative_sample_size': 1024, 
    'hidden_dim': 200, 
    'gamma': 6.0, 
    'negative_adversarial_sampling': True, 
    'adversarial_temperature': 0.5, 
    'batch_size': 512, 
    'regularization': 0.0, 
    'test_batch_size': 8, 
    'uni_weight': False, 
    'learning_rate': 0.0005, 
    'cpu_num': 10, 
    'init_checkpoint': None, 
    'max_steps': 30000, ## 根据经验 RotatE wn18rr 30000
    'warm_up_steps': 20000, 
    'no_save': False, 
    'save_path': 'models/RotatE_wn18rr_baseline', 
    'comments': '\n', 
    'save_checkpoint_steps': 10000, 
    'valid_steps': 10000, 
    'log_steps': 2000, 
    'classify_steps': 5000, 
    'test_log_steps': 1000, 
    'nentity': 40943, 
    'nrelation': 11
}
log_filename = "train"
set_logger(args_rotate_wn, filename=log_filename)
input_data_rotate_wn = get_input_data(args_rotate_wn)
trainer_rotate_wn = BaseTrainer.get_trainer(input_data_rotate_wn, args_rotate_wn)
kge_model_rotate_wn = trainer_rotate_wn.kge_model
logging.info('Start Training...')
logging.info(f"args is {args_rotate_wn}")
logging.info('learning_rate = %f' % trainer_rotate_wn.lr)
for step in range(args_rotate_wn['max_steps']):
    trainer_rotate_wn.basicTrainStep(step)
trainer_rotate_wn.save_model()

## 验证集
logging.info('Evaluating on Valid Dataset...')
metrics = kge_model_rotate_wn.test_step(kge_model_rotate_wn, input_data_rotate_wn.valid_triples, input_data_rotate_wn.all_true_triples, args_rotate_wn)
log_metrics('Valid', args_rotate_wn['max_steps'], metrics)

## 测试集
logging.info('Evaluating on Test Dataset...')
metrics_rotate_wn = kge_model_rotate_wn.test_step(kge_model_rotate_wn, input_data_rotate_wn.test_triples, input_data_rotate_wn.all_true_triples, args_rotate_wn)
log_metrics('Test', args_rotate_wn['max_steps'], metrics_rotate_wn)

2023-12-19 20:03:47,701 INFO     

2023-12-19 20:03:47,701 INFO     

2023-12-19 20:03:47,701 INFO     

2023-12-19 20:03:47,704 INFO     Model: RotatE
2023-12-19 20:03:47,704 INFO     Model: RotatE
2023-12-19 20:03:47,704 INFO     Model: RotatE
2023-12-19 20:03:47,705 INFO     Data Path: data/wn18rr
2023-12-19 20:03:47,705 INFO     Data Path: data/wn18rr
2023-12-19 20:03:47,705 INFO     Data Path: data/wn18rr
2023-12-19 20:03:47,707 INFO     #entity: 40943
2023-12-19 20:03:47,707 INFO     #entity: 40943
2023-12-19 20:03:47,707 INFO     #entity: 40943
2023-12-19 20:03:47,708 INFO     #relation: 11
2023-12-19 20:03:47,708 INFO     #relation: 11
2023-12-19 20:03:47,708 INFO     #relation: 11
2023-12-19 20:03:47,710 INFO     #train: 86835	#valid: 3034	#test: 3134
2023-12-19 20:03:47,710 INFO     #train: 86835	#valid: 3034	#test: 3134
2023-12-19 20:03:47,710 INFO     #train: 86835	#valid: 3034	#test: 3134
2023-12-19 20:03:49,124 INFO     Model Parameter Configuration:
2023-12-19 20:03:49,1

# 4. 开始训练模型 rotate fb15k-237

In [5]:
args_rotate_fb={
    'cuda': True, 
    'fake': None, 
    'do_train': True, 
    'do_valid': False, 
    'do_test': True, 
    'evaluate_train': False, 
    'data_path': 'data/FB15k-237', 
    'model': 'RotatE', 
    'double_entity_embedding': True, 
    'double_relation_embedding': False, 
    'negative_sample_size': 256, 
    'hidden_dim': 200, 
    'gamma': 9.0, 
    'negative_adversarial_sampling': True, 
    'adversarial_temperature': 1.0, 
    'batch_size': 1024, 
    'regularization': 0.0, 
    'test_batch_size': 16, 
    'uni_weight': False, 
    'learning_rate': 0.0005, 
    'cpu_num': 10, 
    'init_checkpoint': None, 
    'max_steps': 30000, ## 根据经验RotatE FB15k-237 30000
    'warm_up_steps': 20000, 
    'no_save': False, 
    'save_path': 'models/RotatE_FB15k-237_baseline', 
    'comments': '\n', 
    'save_checkpoint_steps': 10000, 
    'valid_steps': 10000, 
    'log_steps': 2000, 
    'classify_steps': 5000, 
    'test_log_steps': 1000, 
    'nentity': 14541, 
    'nrelation': 237
}
log_filename = "train"
set_logger(args_rotate_fb, filename=log_filename)
input_data_rotate_fb = get_input_data(args_rotate_fb)
trainer_rotate_fb = BaseTrainer.get_trainer(input_data_rotate_fb, args_rotate_fb)
kge_model_rotate_fb = trainer_rotate_fb.kge_model
logging.info('Start Training...')
logging.info(f"args is {args_rotate_fb}")
logging.info('learning_rate = %f' % trainer_rotate_fb.lr)
for step in range(args_rotate_fb['max_steps']):
    trainer_rotate_fb.basicTrainStep(step)
trainer_rotate_fb.save_model()

## 验证集
logging.info('Evaluating on Valid Dataset...')
metrics = kge_model_rotate_fb.test_step(kge_model_rotate_fb, input_data_rotate_fb.valid_triples, input_data_rotate_fb.all_true_triples, args_rotate_fb)
log_metrics('Valid', args_rotate_fb['max_steps'], metrics)

## 测试集
logging.info('Evaluating on Test Dataset...')
metrics_rotate_fb = kge_model_rotate_fb.test_step(kge_model_rotate_fb, input_data_rotate_fb.test_triples, input_data_rotate_fb.all_true_triples, args_rotate_fb)
log_metrics('Test', args_rotate_fb['max_steps'], metrics_rotate_fb)

2023-12-19 20:05:29,037 INFO     

2023-12-19 20:05:29,037 INFO     

2023-12-19 20:05:29,037 INFO     

2023-12-19 20:05:29,037 INFO     

2023-12-19 20:05:29,041 INFO     Model: RotatE
2023-12-19 20:05:29,041 INFO     Model: RotatE
2023-12-19 20:05:29,041 INFO     Model: RotatE
2023-12-19 20:05:29,041 INFO     Model: RotatE
2023-12-19 20:05:29,045 INFO     Data Path: data/FB15k-237
2023-12-19 20:05:29,045 INFO     Data Path: data/FB15k-237
2023-12-19 20:05:29,045 INFO     Data Path: data/FB15k-237
2023-12-19 20:05:29,045 INFO     Data Path: data/FB15k-237
2023-12-19 20:05:29,050 INFO     #entity: 14541
2023-12-19 20:05:29,050 INFO     #entity: 14541
2023-12-19 20:05:29,050 INFO     #entity: 14541
2023-12-19 20:05:29,050 INFO     #entity: 14541
2023-12-19 20:05:29,052 INFO     #relation: 237
2023-12-19 20:05:29,052 INFO     #relation: 237
2023-12-19 20:05:29,052 INFO     #relation: 237
2023-12-19 20:05:29,052 INFO     #relation: 237
2023-12-19 20:05:29,055 INFO     #train: 272115	#val